In [1]:
!pip3 install requests Pillow numpy opencv-python python-dotenv

  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_x86_64.whl.metadata (19 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 2.1 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 14.6 MB/s  0:00:03m0:00:0100:01
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [opencv-python]0m [opencv-python]


In [1]:


import os
from dotenv import load_dotenv
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import json
from IPython.display import display, HTML


load_dotenv() 
PEXELS_API_KEY = os.getenv("PEXELS_API_KEY")

if not PEXELS_API_KEY:
    raise ValueError("PEXELS_API_KEY not found. Please create a .env file with your Pexels API key.")


keyword = input("Enter search keyword: ")
num_photos = int(input("Enter number of photos to fetch: "))


def fetch_pexels_images(query, per_page=5):
    headers = {"Authorization": PEXELS_API_KEY}
    url = f"https://api.pexels.com/v1/search?query={query}&per_page={per_page}"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print("Error fetching Pexels API:", resp.status_code, resp.text)
        return []

    data = resp.json()
    if "photos" not in data:
        print("Unexpected API response:", data)
        return []
    images = [{"id": p["id"], "url": p["src"]["medium"], "name": p["photographer"]} for p in data["photos"]]
    return images

images = fetch_pexels_images(keyword, num_photos)


def colorfulness_score(url):
    resp = requests.get(url)
    img = Image.open(BytesIO(resp.content)).convert("RGB")
    img_np = np.array(img)
    (R, G, B) = img_np[:,:,0], img_np[:,:,1], img_np[:,:,2]
    rg = np.abs(R - G)
    yb = np.abs(0.5*(R + G) - B)
    return np.mean(np.sqrt(rg**2 + yb**2))

def resolution_score(url):
    resp = requests.get(url)
    img = Image.open(BytesIO(resp.content))
    return img.width * img.height

for img in images:
    img["resolution"] = resolution_score(img["url"])
    img["colorfulness"] = colorfulness_score(img["url"])
    img["score"] = img["resolution"]*0.5 + img["colorfulness"]*0.5

images_sorted = sorted(images, key=lambda x: x["score"], reverse=True)


chart_data = [["Image Name", "Score"]] + [[img["name"], img["score"]] for img in images_sorted]


html_chart = f"""
<html>
<head>
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
<script type="text/javascript">
google.charts.load('current', {{packages:['corechart']}});
google.charts.setOnLoadCallback(drawChart);
function drawChart() {{
  var data = google.visualization.arrayToDataTable({json.dumps(chart_data)});
  var options = {{
    title: 'Top {keyword.capitalize()} Images',
    legend: {{ position: 'none' }},
    chartArea: {{ width: '80%', height: '70%' }}
  }};
  var chart = new google.visualization.BarChart(document.getElementById('chart_div'));
  chart.draw(data, options);
}}
</script>
</head>
<body>
<h2>Top {keyword.capitalize()} Images</h2>
<div id="chart_div" style="width:900px; height:500px;"></div>
<h3>Top Images</h3>
<div style="display:flex; flex-wrap:wrap;">
"""
for img in images_sorted:
    html_chart += f'<div style="margin:5px;text-align:center;"><img src="{img["url"]}" width="150"><br>{img["name"]}</div>'

top_img = images_sorted[0]
html_chart += f"""
</div>
<p>Download top image: <a href="{top_img['url']}" download>Click here</a></p>
</body>
</html>
"""

display(HTML(html_chart))

